In [32]:
import pandas as pd
import numpy as np

In [33]:
cars = pd.read_csv('../data/cars.csv')
cars.head(3)

,Unnamed: 0,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,Unnamed: 16
0,449069,2006,Chrysler,300,Base,Sedan,automatic,2c3ka43r76h366165,il,1.9,119618.0,blue,gray,go financial,4775,2500,Thu May 28 2015 03:00:00 GMT-0700 (PDT),NaN
1,197199,2014,Volkswagen,Jetta,S,Sedan,manual,3vw1k7aj9em275429,ca,4.9,7740.0,—,black,volkswagen credit inc/sc,12050,12750,Wed Jul 01 2015 09:30:00 GMT-0700 (PDT),NaN
2,288171,2007,Dodge,Durango,SLT,SUV,automatic,1d8hd48p97f510727,fl,1.9,166628.0,gold,tan,mid atlantic finance/clearwater,2850,2700,Wed Feb 11 2015 08:00:00 GMT-0800 (PST),NaN


In [34]:
cars.dtypes

Unnamed: 0        int64
year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr               int64
sellingprice      int64
saledate         object
Unnamed: 16     float64
dtype: object

# Ordinal Encoder

In [35]:
datos = [
    ['alto', 'rojo'],
    ['bajo', 'verde'],
    ['medio', 'azul'],
    ['alto', 'verde']
]

In [36]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [37]:
ordinal_encoder.fit(datos)

OrdinalEncoder()

In [38]:
codificados = ordinal_encoder.transform(datos)
codificados

array([[0., 1.],
       [1., 2.],
       [2., 0.],
       [0., 2.]])

In [39]:
ordinal_encoder.categories_

[array(['alto', 'bajo', 'medio'], dtype=object),
 array(['azul', 'rojo', 'verde'], dtype=object)]

# Codificacion de etiquetas

In [40]:
cars['color'].nunique()

19

In [41]:
cars['color'].value_counts()[:5]

color
black     2015
white     1931
gray      1506
silver    1503
blue       869
Name: count, dtype: int64

In [42]:
cars['color'] = cars['color'].astype('category')
cars['color'].dtype

CategoricalDtype(categories=['beige', 'black', 'blue', 'brown', 'burgundy', 'charcoal',
                  'gold', 'gray', 'green', 'off-white', 'orange', 'pink',
                  'purple', 'red', 'silver', 'turquoise', 'white', 'yellow',
                  '—'],
, ordered=False)

In [43]:
cars['color_enc'] = cars['color'].cat.codes
cars['color_enc'].value_counts()[:5]

color_enc
1     2015
16    1931
7     1506
14    1503
2      869
Name: count, dtype: int64

## With Scikit-learn

In [44]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [45]:
# cars['color'] = encoder.fit_transform(cars['color'])
# cars['color'].value_counts()[:5]

In [46]:
encoder.fit(cars['color'])
cars['color_enc'] = encoder.transform(cars['color'])

In [47]:
cars['color'].value_counts()[:5]

color
black     2015
white     1931
gray      1506
silver    1503
blue       869
Name: count, dtype: int64

In [48]:
encoder.inverse_transform([2])

array(['blue'], dtype=object)

### Test

In [49]:
datos = ['etiqueta1', 'etiqueta2', 'etiqueta3', 'etiqueta2', 'etiqueta1']
encoder2 = LabelEncoder()
encoder2.fit(datos)

LabelEncoder()

In [50]:
codificados = encoder2.transform(datos)
codificados

array([0, 1, 2, 1, 0])

In [51]:
encoder2.inverse_transform(codificados)

array(['etiqueta1', 'etiqueta2', 'etiqueta3', 'etiqueta2', 'etiqueta1'],
      dtype='<U9')

# One Hot Encoding

In [52]:
cars['color'][:5]

0     blue
1        —
2     gold
3    white
4      red
Name: color, dtype: category
Categories (19, object): ['beige', 'black', 'blue', 'brown', ..., 'turquoise', 'white', 'yellow', '—']

In [53]:
ohe = pd.get_dummies(cars['color'])

In [54]:
ohe[:5]

,beige,black,blue,brown,burgundy,charcoal,gold,gray,green,off-white,orange,pink,purple,red,silver,turquoise,white,yellow,—
0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


# Codificacion binaria

In [55]:
from category_encoders import BinaryEncoder

encoder = BinaryEncoder(cols=['color'], drop_invariant=True).fit(cars['color'])
encoder.transform(cars['color'])[:5]

,color_0,color_1,color_2,color_3,color_4
0,0,0,0,0,1
1,0,0,0,1,0
2,0,0,0,1,1
3,0,0,1,0,0
4,0,0,1,0,1


# Hashing

Otra opción que tenemos disponible es una técnica de codificación llamada hashing. Este proceso es similar a la codificación de un solo disparo donde creará nuevas columnas binarias, pero dentro de los parámetros, puede decidir cuántas funciones generar. Una gran ventaja es la dimensionalidad reducida, pero una gran desventaja es que algunas categorías se asignarán a los mismos valores. Eso se llama colisión.

In [56]:
from category_encoders import HashingEncoder

encoder = HashingEncoder(cols=['color'], n_components=4).fit(cars['color'])
hash_result = encoder.transform(cars['color'])
hash_result[:5]

,col_0,col_1,col_2,col_3
0,0,0,0,1
1,0,0,0,1
2,0,1,0,0
3,1,0,0,0
4,1,0,0,0


Ahora puedes estar pensando, ¿Cuándo usaría esto si voy a perder información y mi modelo verá marrón y carbón ( o algún otro combo de color con el mismo valor hash ) que la misma cosa? Bueno, esto podría ser una solución para su proyecto y conjunto de datos si no está tan interesado en evaluar el impacto de un valor categórico en particular.

Para este ejemplo, tal vez no esté interesado en saber qué color de automóvil tuvo un impacto en su predicción final, pero desea obtener el mejor rendimiento de su modelo. Esta solución de codificación puede ser un buen enfoque.

# Target encoding

La codificación de destino es un codificador bayesiano utilizado para transformar características categóricas en valores numéricos hash y a veces se denomina codificador medio. Este codificador se puede utilizar para conjuntos de datos que se están preparando para el aprendizaje supervisado basado en regresión, ya que debe tener en cuenta la media de la variable objetivo y su correlación entre cada categoría individual de nuestra característica. De hecho, los valores numéricos de cada categoría se reemplazan con una combinación de la probabilidad posterior del objetivo dado un valor categórico particular y la probabilidad previa del objetivo sobre todos los datos de entrenamiento.

In [57]:
from category_encoders import TargetEncoder

encoder = TargetEncoder(cols=['color'])

In [58]:
encoder_result = encoder.fit_transform(cars['color'], cars['sellingprice'])
encoder_result.head(5)

,color
0,11761.881473
1,18007.276995
2,8458.251290
3,14769.292595
4,12691.099747


Podemos examinar todos los diferentes valores que posee nuestro codificador_resultados, y si miramos la salida desde abajo podemos ver que nuestro mínimo es de aproximadamente 3,054 y nuestro máximo es de aproximadamente 18,048. ¡Esa es una gran diferencia!

In [59]:
np.sort(encoder_result['color'].unique())

array([ 8088.87455397,  8458.25129014,  9276.78734931, 11328.52809902,
       11683.94200687, 11760.1648    , 11761.88147296, 11805.06187625,
       12124.83756218, 12271.29332903, 12691.09974747, 12921.89523399,
       12982.62063008, 13912.83399734, 14354.51705515, 14750.97847138,
       14769.29259451, 15496.72704715, 17174.16914287, 18007.27699531])